In [1]:
import numpy as np
import pandas as pd

In [2]:
#1st - read in the csv file
data = pd.read_csv("C:/Users/Anthony DePinho/Downloads/smallbussample/MBTA Bus Routes/routesample.csv")
#pull out only the shape column which are all in string format
shape_col = data["SHAPE"]
print (shape_col.shape)

(19,)


In [3]:
#create multigon finder and deal with it separately, then .apply(get_coord)
def find_multi(row):
    row_clean = row[16:-3]
    found = False
    if '), (' in row_clean:
        found = True
    return found

In [4]:
#PREP FUNCTION COMES BEFORE FEEDING INTO THE APPROPRIATE MULTI/POLY FUNCTION TO MAKE DATA UNIFORM FORMAT
def prep_row(row):
    #pull off 'MULTIPOLYGON ((' leaving one set of parentheses around string
    cleaned = row[18:-2]
    return cleaned # should look like '(lon lat, lon lat,...), (lon lat, lon lat...), (.....)'

In [5]:
#GET_POLY FUNCTION CLEANS AND DEALS WITH SINGLE POLYGONS
#MULTIPOLYGON ROWS GET FED INTO GET_MULTI, WHICH THEN UTILIZES THIS FUNCTION WITHIN IT
def get_poly(data):
    # Remove first and last parentheses, leaving behind only values separated by commas as a string
#     vertex = data[1:-2] 
    # split the string into elements of a list by each comma
    vertex = data.split(",")
    # strip of white spaces on each element
    vertex = [item.strip() for item in vertex ]
    # format the elements from 'latitude longitude' string to (latitude, longitude) floats
    vertex = [(float(item.split(" ")[0]) ,float(item.split(" ")[1])) for item in vertex]
    return (vertex)

In [6]:
#SCRATCH CODE THAT HAS BEEN REMOVED        
# now put tuples in multigon

#print (get_multi(vertex[16]))


#     vertex = data.split(" ")[1:]
#     print vertex[1::2]
#     first = vertex[0].split("(")[3]
#     last = vertex[-1].split(")")[0]
#     body = vertex[1:-1].split(",")
#     vertex = [first] + vertex[1:-1] + [last]
#     print vertex
#     print map(float, vertex)
#     y = map(float, vertex)
#     y = zip(*[iter(y)]*2)
    
#     print y

#print get_coord(vertex)
# final = vertex.apply(get_coord)
# print get_coord(vertex[16])

In [7]:
#SCALE FUNCTION
def scale(num):
    #create 14 digits by padding 0's if needed
    num1 = str(abs(num[0]))
    num2 = str(abs(num[1]))
    
    if len(num1) < 15:
        num1 = num1.ljust(15, '0')
    elif len(num1) > 15:
        num1 = num1[:15]
    
    if len(num2) < 15:
        num2 = num2.ljust(15, '0')
    elif len(num2) > 15:
        num2 = num2[:15]
    
    #shift decimal for 8 decimal places
    return np.array([-1.*(float(num1) * (10**5)),-1.*(float(num2) * (10**5))])

In [8]:
#CENTER FUNCTION
def center(coord):
    # coord is an np array containing lat and long
    
    #find mean
    mean = coord.mean()
    
    #center by subtracting the mean
    coord = coord - mean
    return coord

In [9]:
#FINDING RANDOM POINTS
def find_randpoints(ranges1, ranges2):
    #ranges is an array of either lat or long
    point1 = np.random.uniform(ranges1[0], ranges1[1], 4)
    point2 = np.random.uniform(ranges2[0], ranges2[1], 4)
    return point1, point2

#6th - Determine where the random points lie in which boundaries (using package)

#7th - Set a percentage and determine point location

In [10]:
#Testing
# (long upper, long lower)
coord1 = np.array([-71.138337, -71.145313])
#(lat upper, lat lower)
coord2 = np.array([42.363305, 42.360741 ])

lon_scaled = scale(coord1)
lon_centered = center(lon_scaled)

lat_scaled = scale(coord2)
lat_centered = center(lat_scaled)

# print find_randpoints(lon_centered, lat_centered)
# print lon_centered[0],lon_centered[1]
# print lat_centered[0], lat_centered[0]

# print find_randpoints(coord1, coord2)
rand_pt1, rand_pt2 = find_randpoints(coord1, coord2)
# print scale(rand_pt1)
scaled1 = scale(rand_pt1)
# print center(scaled1)

# print rand_pt2
# print scale(rand_pt2)
scaled2 = scale(rand_pt2)
# print center(scaled2)

In [11]:

def parse_shapes(one_row):
    is_multi = find_multi(one_row)
    if is_multi == True:
        prepped = prep_row(one_row)
        prepd_split = get_multi(prepped)
        return (prepd_split)
    else:
        prepped = prep_row(one_row)
        prepd_cleand = get_poly(prepped)
        return (prepd_cleand)

In [12]:
print (parse_shapes(shape_col[0]))

[(-71.10067149990927, 42.36180460936539), (-71.10078338044023, 42.36187058192927), (-71.10021868213705, 42.36238285846253), (-71.10065147761732, 42.36266406879039), (-71.10153080626095, 42.363180844275675), (-71.10193201011964, 42.36341799451467), (-71.10306340632803, 42.36409481272895), (-71.10318014937575, 42.364164649313814), (-71.10441851012173, 42.36491304738123), (-71.10458821036588, 42.36501560370796), (-71.1046874097469, 42.36507783581974), (-71.10486397059871, 42.365174660678555), (-71.10494441632486, 42.36521320911077), (-71.10499419785101, 42.36525306743118), (-71.10564871799184, 42.365204365251486), (-71.10615893777172, 42.3651498455367), (-71.10630845925326, 42.365142479917274), (-71.10694213589973, 42.36509287797537), (-71.1074334826154, 42.36504059825034), (-71.10770556733578, 42.36501164786298), (-71.10812987245626, 42.36498495760998), (-71.10890338210636, 42.36491785958375), (-71.10901619049628, 42.36490807347462), (-71.109221007849, 42.36488513701052), (-71.1098922859

In [13]:
# #RESHAPE SHAPE_COL SO WE CAN USE .APPLY FUNCTION
# shape_col = np.reshape(shape_col, (23,1))
# print (shape_col.shape)

In [14]:
# shape_data = pd.DataFrame(shape_col)
# print (shape_data.dtypes)

In [15]:
# #APPLY PARSE_SHAPES METHOD TO ALL ELEMENTS OF SHAPE_COL LIST
# shape_data = shape_data.apply(parse_shapes)

In [16]:
from shapely.geometry import LineString
#Going to hold multiple LineStrings
lines = []
for z in range(0, len(shape_col)):
    lines.append(LineString(parse_shapes(shape_col[z])))
    
print(lines[0].type)
print(lines[0])

LineString
LINESTRING (-71.10067149990927 42.36180460936539, -71.10078338044023 42.36187058192927, -71.10021868213705 42.36238285846253, -71.10065147761732 42.36266406879039, -71.10153080626095 42.36318084427567, -71.10193201011964 42.36341799451467, -71.10306340632803 42.36409481272895, -71.10318014937575 42.36416464931381, -71.10441851012173 42.36491304738123, -71.10458821036588 42.36501560370796, -71.1046874097469 42.36507783581974, -71.10486397059871 42.36517466067856, -71.10494441632486 42.36521320911077, -71.10499419785101 42.36525306743118, -71.10564871799184 42.36520436525149, -71.10615893777172 42.3651498455367, -71.10630845925326 42.36514247991727, -71.10694213589973 42.36509287797537, -71.10743348261541 42.36504059825034, -71.10770556733578 42.36501164786298, -71.10812987245626 42.36498495760998, -71.10890338210636 42.36491785958375, -71.10901619049628 42.36490807347462, -71.109221007849 42.36488513701052, -71.10989228593169 42.36482721179014, -71.11064404193844 42.364766390

In [19]:
#list of coordinates of the first member of linestring
listofcoords0 = list(lines[0].coords)
print(listofcoords0)
print(len(listofcoords0))

[(-71.10067149990927, 42.36180460936539), (-71.10078338044023, 42.36187058192927), (-71.10021868213705, 42.36238285846253), (-71.10065147761732, 42.36266406879039), (-71.10153080626095, 42.363180844275675), (-71.10193201011964, 42.36341799451467), (-71.10306340632803, 42.36409481272895), (-71.10318014937575, 42.364164649313814), (-71.10441851012173, 42.36491304738123), (-71.10458821036588, 42.36501560370796), (-71.1046874097469, 42.36507783581974), (-71.10486397059871, 42.365174660678555), (-71.10494441632486, 42.36521320911077), (-71.10499419785101, 42.36525306743118), (-71.10564871799184, 42.365204365251486), (-71.10615893777172, 42.3651498455367), (-71.10630845925326, 42.365142479917274), (-71.10694213589973, 42.36509287797537), (-71.1074334826154, 42.36504059825034), (-71.10770556733578, 42.36501164786298), (-71.10812987245626, 42.36498495760998), (-71.10890338210636, 42.36491785958375), (-71.10901619049628, 42.36490807347462), (-71.109221007849, 42.36488513701052), (-71.1098922859

In [21]:
print(listofcoords0[0][0])

-71.10067149990927
